In [25]:
import os
from dotenv import load_dotenv
from langchain_community.utilities.tavily_search import TAVILY_API_URL

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[:2])
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
print(TAVILY_API_KEY[:2])

sk
up
tv


In [28]:
import re
from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document
from langchain_upstage import UpstageEmbeddings
from langchain_community.vectorstores import Chroma

# --- 1. 문서 로드 ---
loader = TextLoader("../data/cafe_menu_data.txt", encoding="utf-8")
documents = loader.load()
print("==> 1. cafe_menu.txt 로드 완료")

# --- 2. 문서 분할 ---
# 각 메뉴 항목을 별도의 Document 객체로 분할하는 함수
def split_menu_items(document):
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    return menu_documents

menu_docs = split_menu_items(documents[0])
print(f"==> 2. 총 {len(menu_docs)}개의 메뉴 항목으로 분할 완료")


==> 1. cafe_menu.txt 로드 완료
==> 2. 총 10개의 메뉴 항목으로 분할 완료


In [32]:
# --- 3. 임베딩 모델 설정 ---
embeddings_model = UpstageEmbeddings(model="solar-embedding-1-large")
print("==> 3. Upstage 임베딩 모델 준비 완료")

# --- 4. Chroma 벡터 DB 생성 및 저장 ---
db_dir = "./db/cafe_db"
cafe_db = Chroma.from_documents(
    documents=menu_docs,
    embedding=embeddings_model,
    persist_directory=db_dir
)
print("==> 4. Chroma 벡터 DB 생성 및 저장 완료")

# --- 5. 검색기(Retriever) 테스트 ---
retriever = cafe_db.as_retriever(search_kwargs={"k": 2})
retrieved_docs = retriever.invoke("라떼에 대해 알려줘")
print("\n--- 검색기 테스트 결과 ---")
for doc in retrieved_docs:
    print(doc.metadata)

print("==> 5. 검색기 테스트 완료")

==> 3. Upstage 임베딩 모델 준비 완료
==> 4. Chroma 벡터 DB 생성 및 저장 완료

--- 검색기 테스트 결과 ---
{'menu_name': '카페라떼', 'menu_number': 2, 'source': '../data/cafe_menu_data.txt'}
{'menu_name': '카페라떼', 'menu_number': 2, 'source': '../data/cafe_menu_data.txt'}
==> 5. 검색기 테스트 완료


In [ ]:

# (1) 도구 정의: tavily_search_func
from langchain_community.tools import TavilySearchResults

@tool
def tavily_search_func(query: str) -> str:
    """
    최신 정보나 데이터베이스에 없는 정보를 인터넷에서 검색할 때 사용합니다.
    예를 들어, 최신 커피 트렌드나 특정 카페의 위치 정보 등에 유용합니다.
    """
    tavily_search = TavilySearchResults(max_results=3)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
    ])

    if formatted_docs:
        return formatted_docs

    return "관련 정보를 찾을 수 없습니다."

print("==> 6-1. tavily_search_func 정의 완료")

# (2) 도구 정의: wiki_summary (LCEL 체인을 도구로 변환)
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WikipediaLoader
from langchain_upstage import ChatUpstage
from pydantic import BaseModel, Field
from textwrap import dedent

# WikipediaLoader를 사용하는 함수 정의
def search_wiki_func(input_data: dict) -> List[Document]:
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    return wiki_loader.load()

# 요약 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_template("다음 텍스트를 간결하게 요약해 주세요:\n\n{context}\n\n요약:")

# LLM 초기화
llm = ChatUpstage(model="solar-pro", temperature=0.1)

# 요약 체인 생성
summary_chain = (
    {"context": RunnableLambda(search_wiki_func)}
    | summary_prompt | llm | StrOutputParser()
)

# 도구 입력 스키마 정의
class WikiSummarySchema(BaseModel):
    query: str = Field(..., description="위키피디아에서 검색할 주제")

# Runnable 체인을 도구로 변환
wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        일반적인 지식이나 배경 정보가 필요할 때 위키피디아에서 정보를 검색하고 요약합니다.
        예를 들어, 커피의 역사, 음료 제조 방법 등에 유용합니다.
    """),
    args_schema=WikiSummarySchema
    )

print("==> 6-2. search_wiki_func 정의 완료")

# (3) 도구 정의: db_search_cafe_func
from typing import List
from langchain_core.tools import tool

# 미리 생성된 Chroma DB를 로드합니다.
cafe_db = Chroma(
    embedding_function=embeddings_model,
    persist_directory=db_dir
)

@tool
def db_search_cafe_func(query: str) -> List[Document]:
    """
    로컬 카페 메뉴 데이터베이스에서 정보를 검색할 때 사용합니다.
    메뉴의 가격, 재료, 설명 등과 관련된 질문에 유용합니다.
    """
    docs = cafe_db.similarity_search(query, k=4)
    if docs:
        return docs
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

print("==> 6-3. db_search_cafe_func 정의 완료")

==> 6-1. tavily_search_func 정의 완료
==> 6-2. search_wiki_func 정의 완료
==> 6-3. db_search_cafe_func 정의 완료


In [42]:
# 7. LLM에 모든 도구 바인딩

tools = [db_search_cafe_func, tavily_search_func, wiki_summary]
llm_with_tools = llm.bind_tools(tools=tools)

print("==> 7. LLM 도구 바인딩 완료")
print("3개의 도구가 LLM에 성공적으로 바인딩되었습니다.")
print(f" - 바인딩된 도구: {[tool.name for tool in tools]}")


==> 7. LLM 도구 바인딩 완료
3개의 도구가 LLM에 성공적으로 바인딩되었습니다.
 - 바인딩된 도구: ['db_search_cafe_func', 'tavily_search_func', 'wiki_summary']


In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.runnables import RunnableConfig, chain
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 카페 메뉴와 음식에 대한 지식을 갖춘 AI 어시스턴트입니다. 사용자의 질문에 답하기 위해 주어진 도구를 적절히 활용하세요."),
    ("human", "{user_input}"),
    MessagesPlaceholder(variable_name="messages"),  # 도구 실행 결과 자리
])

llm_chain = prompt | llm_with_tools

@chain
def cafe_tool_chain(user_input: str, config: RunnableConfig):
    # 초기 입력
    input_ = {"user_input": user_input, "messages": []}

    # LLM 호출 → 도구 사용 결정
    ai_msg = llm_chain.invoke(input_, config=config)

    # 도구 실행
    tool_msgs = []
    if ai_msg.tool_calls:
        for tool_call in ai_msg.tool_calls:
            print(f"▶️ 도구 호출: {tool_call['name']}({tool_call['args']})")
            if tool_call["name"] == "db_search_cafe_func":
                tool_output = db_search_cafe_func.invoke(tool_call, config=config)
            elif tool_call["name"] == "tavily_search_func":
                tool_output = tavily_search_func.invoke(tool_call, config=config)
            elif tool_call["name"] == "wiki_summary":
                tool_output = wiki_summary.invoke(tool_call, config=config)

            tool_msgs.append(ToolMessage(content=str(tool_output), tool_call_id=tool_call['id']))

    # 도구 결과 포함 → 최종 답변 생성
    input_["messages"].extend([ai_msg, *tool_msgs])
    return llm_chain.invoke(input_, config=config)

print("==> 8. @chain 데코레이터를 사용한 도구 호출 체인 구현 완료!")


==> 8. @chain 데코레이터를 사용한 도구 호출 체인 구현 완료!


In [44]:
query = "카페라테의 가격과 특징은 무엇인가요?"
response = cafe_tool_chain.invoke(query)

print("\n==> 9. 최종 답변 ")
print(response.content)

▶️ 도구 호출: db_search_cafe_func({'query': '카페라테 가격 특징'})
▶️ 도구 호출: wiki_summary({'query': '카페라테'})

==> 9. 최종 답변 
### 카페라테의 가격과 특징

#### **가격**  
로컬 카페 메뉴 데이터베이스에 따르면, 카페라테의 평균 가격은 **₩5,500**입니다. (지역/카페에 따라 변동 가능)

#### **주요 특징**  
1. **재료**  
   - **에스프레소**와 **스팀 밀크**를 기본으로 합니다.  
   - 우유 거품(폼)은 얇게(약 1cm) 추가되어 부드러운 질감을 강조합니다.  

2. **맛과 질감**  
   - 진한 에스프레소와 우유의 조화로 **크리미하고 부드러운 맛**이 특징입니다.  
   - 카푸치노보다 우유 양이 많아 **덜 진하고 순한 풍미**를 가집니다.  

3. **시각적 요소**  
   - **라테 아트** (우유 거품으로 만드는 예술)로 시각적 즐거움을 제공합니다.  

4. **역사적 배경**  
   - 19세기 이탈리아 가정에서 유래했으며, 1980년대 스타벅스 등으로 대중화되었습니다.  
   - 한국에서는 2000년대 프랜차이즈 커피숍 확산과 함께 인기를 얻었습니다.  

5. **변형 메뉴**  
   - 바닐라 라테, 카페모카, 비건 우유 대체 라테 등 다양한 변형이 존재합니다.  

#### **카푸치노와의 차이점**  
- **우유/거품 비율**: 카페라테는 우유 비중이 높고 거품이 적으며, 카푸치노는 우유 거품이 1/3을 차지합니다.  
- **풍미**: 카페라테는 부드럽고 순한 맛, 카푸치노는 진한 커피 풍미가 강조됩니다.  

> 📌 **참고**: 가격은 카페마다 차이가 있을 수 있으므로, 정확한 정보는 해당 카페에 문의하는 것이 좋습니다.
